# DLT workshop homework

🔹 **Base API URL:**  
```
https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api
```
🔹 **Data format:** Paginated JSON (1,000 records per page).  
🔹 **API Pagination:** Stop when an empty page is returned.

Running below cells in the devcontainer will provide the homework answers

## **Question 1: dlt Version**

We installed dlt with duckdb in the image (see `requirements.txt`)

In [ ]:
import dlt

In [ ]:
dlt.__version__

## **Question 2: Define & Run the Pipeline (NYC Taxi API)**

Use dlt to extract all pages of data from the API.

Steps:

1️⃣ Use the `@dlt.resource` decorator to define the API source.

2️⃣ Implement automatic pagination using dlt's built-in REST client.

3️⃣ Load the extracted data into DuckDB for querying.



How many tables were created?

* 2
* 4
* 6
* 8


In [ ]:

import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator


# Define the API resource for NYC taxi data
@dlt.resource(name="rides")   # <--- The name of the resource (will be used as the table name)
def ny_taxi():
    client = RESTClient(
        base_url="https://us-central1-dlthub-analytics.cloudfunctions.net",
        paginator=PageNumberPaginator(
            base_page=1,
            total_path=None
        )
    )

    for page in client.paginate("data_engineering_zoomcamp_api"):    # <--- API endpoint for retrieving taxi ride data
        yield page   # <--- yield data to manage memory


# define new dlt pipeline
pipeline = dlt.pipeline(pipeline_name="ny_taxi", destination="duckdb", dataset_name="taxi_rides")


# run the pipeline with the new resource
load_info = pipeline.run(ny_taxi, write_disposition="replace")
print(load_info)






In [ ]:
print(pipeline.last_trace)

In [ ]:
# Start a connection to your database using native `duckdb` connection and look what tables were generated:

import duckdb

# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()


## **Question 3: Explore the loaded data**

What is the number of records loaded?



In [ ]:
# explore loaded data
rides_df = pipeline.dataset(dataset_type="default").rides.df()
print(f"The number of loaded records is {len(rides_df)}")

## **Question 4: Trip Duration Analysis**



* Calculate the average trip duration in minutes.


What is the average trip duration?

* 12.3049
* 22.3049
* 32.3049
* 42.3049

In [ ]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM rides;
            """)


res

